# 피라미드 이야기 만들기

In [2]:
! pip install  langchain_ollama
! pip install python-magic

  Using cached langchain_ollama-0.1.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached ollama-0.3.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached langchain_ollama-0.1.1-py3-none-any.whl (12 kB)
Using cached ollama-0.3.1-py3-none-any.whl (10 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
    

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.0 which is incompatible.
unstructured-client 0.25.5 requires idna>=3.4, but you have idna 2.10 which is incompatible.


In [2]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader
from langchain_chroma import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOllama


In [19]:
# system message 
system_message = """
You're an archaeologist studying pyramids
You're exploring the pyramids
i don't need any extra sound effects
Tell me the story in five lines
Monologues are excluded.
"""
system_prompt = SystemMessagePromptTemplate.from_template(system_message)

# human message
human_message = """
처음 입구에서 손전등과 횃불 중 에서 {text1}을 주워서 사용한다.
다음으로 피라미드 내에서 투탕카맨 유물, 클레오파트라 유물, 쿠푸 유물 유물즁 {text2}을 들고단다.
다음으로 피라미내 내에서 투탕카맨 길, 클레오파트라 길, 쿠푸 길 중에서 {text3}이 있는 갈림길을 선택해서 들어가
마지막으로 관을 발견해서 발견한 관뚜껑을 {text4} 
탈출을 시도하는데 {finish}결과를 맞이하는 이야기 만들어줘
"""
human_prompt = HumanMessagePromptTemplate.from_template(human_message)

# 최종 프롬프트 구성
chat_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, # 지정한 역할
     human_prompt]) # 질문

# 모델 설정
model = ChatOllama(model='llama3.1:latest')

# 쓸데없는 정보가 아닌 글자만 가져온다.
outputparser = StrOutputParser()

# 체인 설정
chain = chat_prompt | model | outputparser

result = chain.stream({
    'text1':'손전등',
    'text2':'투탕카맨의 마스크',
    'text3':'투탕카맨 관',
    'text4':'닫고', 'finish':'안좋은'
    })

# for chunk in result:
#     print(chunk, end="", flush=True)
"".join([i for i in result])

'피라미드 입구에 도착했다. 내 손전등을 들고 발등을 찍는 소음만이 있었고, 조명이 밝혀지자 피라미드의 내부가 모습을 드러냈다.\n\n투탕카맨 유물과 클레오파트라 유물을 찾은 후에 쿠푸 유물도 발견하게 되었다. 투탕카맨의 마스크를 든 나의 손이 빛났다.\n\n갈림길에 도달했고, 길들고는 투탕카맨 관이 위치한 것을 알게 되었다. 클레오파트라와 쿠푸의 길을 건너 그곳으로 향한다.\n\n마침내 관을 발견하고서 그 안의 비밀을 들여보니... 그리고는 나도 모르게 관뚜껑을 닫았고 탈출을 시도했지만...\n\n바로 통과가 되지 못했고, 다시 피라미드 안에 갇히는 끔찍한 상황이 찾아왔다.'